In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, ForeignKey, Date, Text, Boolean
from pprint import pprint as pp
from datetime import datetime

from config import local_mysql_password, local_mysql_user, FIPS_10_country_codes

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 200)

## Import FAO crop yield data

In [2]:
fao_file_path = "./data/fao_data/"

# Load the crop yield data
file_name = 'Production_Crops_Livestock_E_All_Data_(Normalized)'
unprocessed_fao_crop_yield_data = pd.read_csv(f"{fao_file_path}fao_crop_data/normalized/{file_name}.csv", encoding='latin-1')

### Preprocessing

In [3]:
fao_crop_yield_data = unprocessed_fao_crop_yield_data.copy()
# Replace spaces column names with underscores and make lower case
fao_crop_yield_data.columns = fao_crop_yield_data.columns.str.replace(' ','_').str.lower()
# Rename confusing data aggregation flags
fao_crop_yield_data.flag.replace(np.nan, 'O', inplace=True)
fao_crop_yield_data.flag.replace('*', 'U', inplace=True)
# Remove 0 crop yield values to prevent bad training data
fao_crop_yield_data = fao_crop_yield_data[fao_crop_yield_data.value != 0.0]
# Drop the year_code because it is always the same as the year value
fao_crop_yield_data.drop('year_code', axis=1, inplace=True)
# Drop areas which are not countries
regions = [
    'World', 'Africa', 'Eastern Africa', 'Middle Africa', 'Northern Africa', 'Southern Africa', 'Western Africa', 'Americas',
    'Northern America', 'Central America', 'Caribbean', 'South America', 'Asia', 'Central Asia', 'Eastern Asia',
    'Southern Asia', 'South-eastern Asia', 'Western Asia', 'Europe', 'Eastern Europe', 'Northern Europe', 'Southern Europe',
    'Western Europe', 'Oceania', 'Australia and New Zealand', 'Melanesia', 'Micronesia', 'Polynesia'
    ]
special_groups = [
    'European Union (28)', 'European Union (27)', 'Least Developed Countries', 'Land Locked Developing Countries', 'Small Island Developing States',
    'Low Income Food Deficit Countries', 'Net Food Importing Developing Countries', 'Annex I countries', 'Non-Annex I countries',  'OECD'
    ]
fao_countries = fao_crop_yield_data.area.loc[~fao_crop_yield_data.area.isin(regions+special_groups)].unique().tolist()
fao_crop_yield_data = fao_crop_yield_data[fao_crop_yield_data.area.isin(fao_countries)]
# Rename columns so they aren't confused with the NOAA data
fao_crop_yield_data.rename({
    'area_code': 'fao_country_code', 
    'area': 'fao_country_name', 
    'flag': 'fao_data_quality_id', 
    'item': 'product', 
    'item_code': 'product_id',
    'element': 'production_type',
    'element_code': 'production_type_id'
    }, axis=1, inplace=True)

fao_crop_yield_data.head()

,fao_country_code,fao_country_name,product_id,product,production_type_id,production_type,year,unit,value,fao_data_quality_id
1,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,1976,ha,5900.0,F
2,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,1977,ha,6000.0,F
3,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,1978,ha,6000.0,F
4,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,1979,ha,6000.0,F
5,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,1980,ha,5800.0,F


In [4]:
fao_crop_yield_data.shape

(2802701, 10)

### Combine UCM, NOAA and FAO country lists to match crop calendars and weather data to crop yields

In [30]:
# Load the fao countries
fao_country_data = fao_crop_yield_data.groupby(['fao_country_name', 'fao_country_code']).year.agg(['min', 'max'])
fao_country_data.reset_index(inplace=True)
# Merge with the FIPS country codes
country_codes = pd.DataFrame(FIPS_10_country_codes)
country_codes = country_codes.merge(fao_country_data, how='inner', left_on='country_name', right_on='fao_country_name')
# Load the noaa ghcnd countries
noaa_file_path = './data/noaa_ghcn_aws_data/'
noaa_countries_file_name = 'ghcnd-countries'
ghcnd_countries = pd.read_fwf(f"{noaa_file_path}{noaa_countries_file_name}.txt", header=None, names = ["noaa_country_code", "noaa_country_name"])
# Join the noaa countries
country_codes = country_codes.merge(ghcnd_countries, how='inner', left_on='noaa_country_code', right_on='noaa_country_code')

country_codes.drop(['noaa_country_name', 'fao_country_name'], axis=1, inplace=True)
country_codes.rename(columns={'min': '', '': '', '': 'min'})
country_codes.head()

,country_name_x,FIPS_10_country_code,noaa_country_code,fao_country_name,fao_country_code,min,max,country_name_y
0,Afghanistan,AF,AF,Afghanistan,2,1961,2020,Afghanistan
1,Albania,AL,AL,Albania,3,1961,2020,Albania
2,Algeria,AG,AG,Algeria,4,1961,2020,Algeria
3,Angola,AO,AO,Angola,7,1961,2020,Angola
4,Antigua and Barbuda,AC,AC,Antigua and Barbuda,8,1961,2020,Antigua and Barbuda


,country_name,FIPS_10_country_code,noaa_country_code,fao_country_name,fao_country_code,min,max
0,Afghanistan,AF,AF,Afghanistan,2,1961,2020
1,Albania,AL,AL,Albania,3,1961,2020
2,Algeria,AG,AG,Algeria,4,1961,2020
3,Angola,AO,AO,Angola,7,1961,2020
4,Antigua and Barbuda,AC,AC,Antigua and Barbuda,8,1961,2020


In [16]:
for country in fao_country_data.fao_country_name.values:
    if country not in FIPS_10_fao_country_codes_df.fao_country_name.values:
        print(country)

Belgium-Luxembourg
China, mainland
Czechoslovakia
Ethiopia PDR
French Guyana
Serbia and Montenegro
Sudan (former)
USSR
Yugoslav SFR


(170, (219, 2))

In [23]:
ghcnd_countries.head()

,noaa_country_code,country_name
0,AC,Antigua and Barbuda
1,AE,United Arab Emirates
2,AF,Afghanistan
3,AG,Algeria
4,AJ,Azerbaijan


country_name_x          0
FIPS_10_country_code    0
noaa_country_code       0
fao_country_name        0
fao_country_code        0
min                     0
max                     0
country_name_y          0
dtype: int64

In [4]:
FIPS_10_country_codes_df[FIPS_10_country_codes_df.ghcnd_country_code.isna()]

,country_name_x,FIPS_10_country_code,noaa_country_code,ghcnd_country_code,country_name_y
1,Aland Islands,-,-,NaN,NaN
2,"Bonaire, Sint Eustatius and Saba",-,-,NaN,NaN
3,United States Minor Outlying Islands,-,-,NaN,NaN
7,Andorra,AN,AN,NaN,NaN
9,Anguilla,AV,AV,NaN,NaN
14,Aruba,AA,AA,NaN,NaN
27,Bhutan,BT,BT,NaN,NaN
31,Bouvet Island,BV,BV,NaN,NaN
50,Comoros,CN,CN,NaN,NaN
62,Djibouti,DJ,DJ,NaN,NaN


TypeError: Can only merge Series or DataFrame objects, a <class 'list'> was passed

In [358]:
fao_to_noaa_country_map = {
    'Bahamas': 'Bahamas, The',
    'Bolivia (Plurinational State of)': 'Bolivia',
    'Brunei Darussalam': 'Brunei',
    'Cabo Verde': 'Cape Verde',
    'Congo': 'Congo (Brazzaville)',
    'Cook Islands': 'Cook Islands [New Zealand]',
    'Czechia': 'Czech Republic',
    "Democratic People's Republic of Korea": 'Korea, North',
    'Democratic Republic of the Congo': 'Congo (Kinshasa)',
    'French Guyana': 'French Guiana [France]',
    'Gambia': 'Gambia, The',
    'Guadeloupe': 'Guadeloupe [France]',
    'Iran (Islamic Republic of)': 'Iran',
    'Martinique': 'Martinique [France]',
    'Micronesia (Federated States of)': 'Federated States of Micronesia',
    'Myanmar': 'Burma',
    'New Caledonia': 'New Caledonia [France]',
    'Niue': 'Niue [New Zealand]',
    'Puerto Rico': 'Puerto Rico [United States]',
    'Republic of Korea': 'Korea, South',
    'Russian Federation': 'Russia',
    'Samoa': 'American Samoa [United States]',
    'Syrian Arab Republic': 'Syria',
    'Tokelau': 'Tokelau [New Zealand]',
    'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
    'United Republic of Tanzania': 'Tanzania',
    'United States of America': 'United States',
    'Venezuela (Bolivarian Republic of)': 'Venezuela',
    'Viet Nam': 'Vietnam',
    'North Macedonia': 'Macedonia',
    'Réunion': 'Reunion [France]',
    "Côte d'Ivoire": "Cote D'Ivoire",
    "China, Macao SAR": "Macau S.A.R",
    "Lao People's Democratic Republic": "Laos",
    "Republic of Moldova": "Moldova",
    }

country_data = []
for fao_country_name, noaa_country_name in fao_to_noaa_country_map.items():
        country_data.append({
            'fao_country_code': fao_country_data[fao_country_data.fao_country_name==fao_country_name].fao_country_code.values[0],
            'fao_country_name': fao_country_name,
            'noaa_country_name': noaa_country_name,
            'noaa_country_code': noaa_countries[noaa_countries.ghcnd_country==noaa_country_name].FIPS_country_code.values[0],
        })

for country in fao_countries:
    if country in noaa_countries.ghcnd_country.tolist() and country not in fao_to_noaa_country_map.keys():
        country_data.append({
            'fao_country_code': fao_country_data[fao_country_data.fao_country_name==country].fao_country_code.values[0],
            'fao_country_name': country,
            'noaa_country_name': country,
            'noaa_country_code': noaa_countries[noaa_countries.ghcnd_country==country].FIPS_country_code.values[0],
        })
    elif country not in fao_to_noaa_country_map.keys():
        country_data.append({
            'fao_country_code': fao_country_data[fao_country_data.fao_country_name==country].fao_country_code.values[0],
            'fao_country_name': country,
            'noaa_country_name': np.nan,
            'noaa_country_code': np.nan,
        })
country_data = pd.DataFrame(country_data).sort_values(['noaa_country_name', 'fao_country_name'])
#country_data.dropna(inplace=True)

eu_countries = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden']
#country_data[(country_data.noaa_country_name.isin(eu_countries)) | (country_data.fao_country_name.isin(eu_countries))]
country_data['EU'] = country_data.noaa_country_name.apply(lambda x: True if x in eu_countries else False)


country_data[country_data.noaa_country_code.isna()]

,fao_country_code,fao_country_name,noaa_country_name,noaa_country_code,EU
50,15,Belgium-Luxembourg,NaN,NaN,False
53,18,Bhutan,NaN,NaN,False
67,96,"China, Hong Kong SAR",NaN,NaN,False
69,214,"China, Taiwan Province of",NaN,NaN,False
68,41,"China, mainland",NaN,NaN,False
71,45,Comoros,NaN,NaN,False
76,51,Czechoslovakia,NaN,NaN,False
78,72,Djibouti,NaN,NaN,False
87,209,Eswatini,NaN,NaN,False
89,62,Ethiopia PDR,NaN,NaN,False


In [364]:
fao_name_to_noaa_code = {
"Belgium-Luxembourg": "BE",
"Bhutan": "BT",
"China, Hong Kong SAR": "HK",
"China, Taiwan Province of": "TW",
"China, mainland": "CH",
"Comoros": "CN",
"Djibouti": "DJ",
"Eswatini": "WZ",
"Ethiopia PDR": "ET",
"Faroe Islands": "FO",
"Grenada": "GJ",
"Haiti": "HA",
"Nauru": "NR",
"Palestine": "WE",
"Saint Kitts and Nevis": "SC",
"Saint Vincent and the Grenadines": "VC",
"Sao Tome and Principe": "TP",
"Serbia and Montenegro": "RI",
"Somalia": "SO",
"South Sudan": "OD",
"Sudan (former)": "SU",
"Timor-Leste": "TT",
"USSR": "TP",
"Yemen": "YM",
"Yugoslav SFR": "",
}
for name, code in fao_name_to_noaa_code.items():
    print(name, code, noaa_countries[noaa_countries.FIPS_country_code==code].ghcnd_country.values)

Belgium-Luxembourg BE ['Belgium']
Bhutan BT []
China, Hong Kong SAR HK []
China, Taiwan Province of TW []
China, mainland CH ['China']
Comoros CN []
Djibouti DJ []
Eswatini WZ ['Swaziland']
Ethiopia PDR ET ['Ethiopia']
Faroe Islands FO []
Grenada GJ []
Haiti HA []
Nauru NR []
Palestine WE []
Saint Kitts and Nevis SC []
Saint Vincent and the Grenadines VC []
Sao Tome and Principe TP []
Serbia and Montenegro RI ['Serbia']
Somalia SO []
South Sudan OD []
Sudan (former) SU ['Sudan']
Timor-Leste TT []
USSR TP []
Yemen YM []
Yugoslav SFR  []


In [ ]:
for country in noaa_countries

## Import University of Wisconson-Madison crop calendar

In [348]:

crop_calendar_file_path = "./data/"

# Load the crop calendar
file_name = 'crop_calendar_uni_wisconsin_madison'
unprocessed_uwm_crop_calendar = pd.read_csv(f"{crop_calendar_file_path}{file_name}.csv")
unprocessed_uwm_crop_calendar.head()

,Unnamed: 0,Data.ID,Location,Level,Nation.code,State.code,County.code,Crop,Qualifier,Crop.name.in.original.data,Plant.start,Plant.start.date,Plant.end,Plant.end.date,Plant.median,Plant.range,Harvest.start,Harvest.start.date,Harvest.end,Harvest.end.date,Harvest.median,Harvest.range,Source,Notes,X,full.crop.name,location.code,tot.days,lat.avg,lon.avg,harvested.area,tmin.day.avg,temp.average,precip.average,temp.min,precip.min,temp.max,precip.max,temp.min.month,precip.min.month,temp.max.month,precip.max.month,temp.at.planting,precip.at.planting,sunfrac.at.planting,daylength.at.planting,dsw.at.planting,precip.over.pet.min,precip.over.pet.min.day,precip.over.pet.max,precip.over.pet.max.day,precip.over.pet.at.planting,apr.to.sept.temp.anomaly,apr.to.sept.precip.frac,gdd.base.0.between.plant.and.harvest,gdd.base.4.between.plant.and.harvest,gdd.base.5.between.plant.and.harvest,gdd.base.8.between.plant.and.harvest,gdd.base.10.between.plant.and.harvest,gdd.base.0,gdd.base.4,gdd.base.5,gdd.base.8,gdd.base.10,gdd.base.0.from.plant.until.cold,gdd.base.4.from.plant.until.cold,gdd.base.5.from.plant.until.cold,gdd.base.8.from.plant.until.cold,gdd.base.10.from.plant.until.cold,ndays.growing.season.below.0,ndays.growing.season.below.5,ndays.growing.season.below.10,ndays.growing.season.below.12,ndays.growing.season.below.15,ndays.growing.season.below.17,ndays.below.0,ndays.below.5,ndays.below.10,ndays.below.12,ndays.below.15,ndays.below.17,LGP,LGP.T,LGP.P,LGP.tmin5.tmax100,LGP.T.tmin5.tmax100,LGP.P.tmin5.tmax100,temp.growing.season,precip.growing.season,ndays.since.snowfrac_0.5,ndays.since.snowfrac_0.1,ndays.since.snowfrac_0.05,ndays.since.snowfrac_0,climate.category,misclassified,climate.category.string
0,1,1,Albania,N,55,NaN,NaN,Wheat,Winter,Wheat,289,10/16,349,12/15,319.0,61,121.0,5/1,181.0,6/30,151.0,61.0,GIEWS,NaN,NaN,Wheat.Winter,55,197.0,40.936144,19.920749,1120.283380,-11.043975,13.298533,93.923606,4.623191,31.523111,22.225507,163.806151,1.0,7.0,7.0,11.0,9.907873,163.806151,0.423464,10.886388,87.250043,0.167999,197.0,6.756057,350.0,4.486921,5.123738,0.616576,1754.963575,966.963575,773.447847,391.263188,238.614476,4869.060207,3409.060207,3047.544478,2161.359819,1672.727825,NaN,NaN,106.211578,14.813273,0.000000,0.0,18.0,137.0,153.0,174.0,190.0,0.0,18.0,138.0,167.0,208.0,237.0,252.0,365.0,252.0,233.0,346.0,252.0,8.908445,113.915234,276.0,219.0,-152.0,-160.0,1.0,True,1X
1,2,2,Albania,N,55,NaN,NaN,Maize,NaN,Maize,106,4/16,167,6/16,136.5,62,228.0,8/16,304.0,10/31,266.0,77.0,GIEWS,NaN,NaN,Maize,55,129.5,41.149095,19.968137,540.683231,-12.522727,12.599007,94.823209,3.682332,35.848386,21.646772,160.831472,1.0,7.0,7.0,11.0,15.691555,76.187750,0.543914,15.586047,244.024139,0.195790,197.0,6.959926,350.0,0.559177,5.238819,0.650312,2555.724795,2035.724795,1905.724795,1515.724795,1255.724795,4613.974357,3156.279807,2828.215926,2000.261356,1535.126257,NaN,2629.199423,2400.734745,1789.673591,1422.172811,0.0,0.0,0.0,0.0,0.0,13.0,0.0,57.0,148.0,176.0,216.0,247.0,112.0,218.0,259.0,202.0,308.0,259.0,19.659421,53.091993,94.0,37.0,31.0,23.0,1.0,False,1
2,3,4,Albania,N,55,NaN,NaN,Pulses,NaN,Beans,106,4/16,177,6/26,141.5,72,197.0,7/16,243.0,8/31,220.0,47.0,GIEWS,NaN,NaN,Pulses,55,78.5,40.981883,19.946543,281.843038,-11.520141,13.016876,93.816973,4.271594,32.871042,21.983251,161.873432,1.0,7.0,7.0,11.0,16.504997,69.337616,0.568113,15.733263,253.247260,0.176631,197.0,6.768559,350.0,0.481453,5.157698,0.627937,1588.814649,1272.814649,1193.814649,956.814649,798.814649,4766.339416,3306.339416,2954.086708,2093.502256,1614.442635,NaN,NaN,2434.024415,1821.777486,1456.814408,0.0,0.0,0.0,0.0,0.0,5.0,0.0,35.0,142.0,171.0,211.0,241.0,112.0,224.0,253.0,218.0,330.0,253.0,20.111578,45.102474,99.0,42.0,36.0,28.0,1.0,False,1
3,4,5,Albania,N,55,NaN,NaN,Other,Winter vegetables,Winter vegetables,289,10/16,340,12/6,314.5,52,45.0,2/14,105.0,4/15,75.0,61.0,GIEWS,"Cabbage, cauliflower, spinach, onion, garlic",NaN,Other.Winter vegetables,55,125.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [350]:
uwm_crop_calendar = unprocessed_uwm_crop_calendar.copy()
uwm_crop_calendar = uwm_crop_calendar[[
    'Location', 'Nation.code', 'Crop', 'Qualifier',
    'Crop.name.in.original.data', 'Plant.start', 'Plant.start.date',
    'Plant.end', 'Plant.end.date', 'Plant.median', 'Plant.range',
    'Harvest.start', 'Harvest.start.date', 'Harvest.end',
    'Harvest.end.date', 'Harvest.median', 'Harvest.range', 'Source',
    'Notes', 'full.crop.name', 'lat.avg',
    'lon.avg']]

In [353]:
uwm_to_noaa_country_map = {
    "Central African Republic (South)": "CT",
    "Congo": "CF",
    "Congo Dem. Rep.": "CG",
    "Congo Dem. Rep. (North)": "CG",
    "Congo Dem. Rep. (Centre)": "CG",
    "Congo Dem. Rep. (South)": "CG",
    "Congo Dem. Rep. (Extr-South)": "CG",
    "North Korea": "KN",
    "North Korea (central bowl)": "KN",
    "Timor Leste": "TT",
    "French Guiana": "FG",
    "The Gambia": "GA",
    "Ghana (South)": "GH",
    "Ghana (North)": "GH",
    "Haiti": "HA",
    "Indonesia (Java)": "ID",
    "Indonesia (South Sulawesi (east))": "ID",
    "Indonesia (South Sulawesi (west))": "ID",
    "Indonesia (Sumatra)": "ID",
    "Cote d'Ivoire": "IV",
    "Malaysia (Sabah prov.)": "MY",
    "Malaysia (Sarawak prov.)": "MY",
    "Myanmar": "BM",
    "Nigeria (South)": "NI",
    "Nigeria (North)": "NI",
    "Somalia": "SO",
    "South Africa (West)": "SF",
    "South Korea": "KS",
    "Sudan (South)": "OD",
    "Sudan (North)": "SU",
    "Tanzania (South)": "TZ",
    "Tanzania (North)": "TZ",
    "Trinidad & Tobago": "TD",
    "Uganda (South)": "UG",
    "Uganda (North)": "UG",
    "Brazil (Center-South)": "BR",
    "Brazil (Rio Grande do Sul)": "BR",
    "Brazil (Northeast)": "BR",
    "Canadian Prairies (Alberta, Saskatchewan, Manitoba)": "CA",
    "Ontario & Quebec": "CA",
    "China (north)": "CH",
    "China (North China Plain)": "CH",
    "China (North China Plain and Manchuria)": "CH",
    "China (south)": "CH",
    "Mexico (not northwest)": "MX",
    "Mexico (NW)": "MX",
    "Brazil (North & Northeast)": "BR",
    "Czechoslovakia": "EZ",
    "Yugoslavia (Former)": "RI",
    "India (North)": "IN",
    "Burma (Myanmar)": "BM",
    "Thailand (central & northern)": "TH",
    "Thailand (NE)": "TH",
    "Vietnam (North)": "VN",
    "Vietnam (South)": "VN",
    "Indonesia (S. Sumatra & E. Java)": "ID",
    "South Africa (East)": "SF",
    "South Africa (except Western Cape)": "SF",
    "China (Sichuan, Hubei and Hunan)": "CH",
    "China (South)": "CH",
    "Mexico (Yucatan)": "MX",
    "Alaska": "US",
    "California": "US",
    "Colorado": "US",
    "Idaho": "US",
    "Kansas": "US",
    "Michigan": "US",
    "Minnesota": "US",
    "Montana": "US",
    "Nebraska": "US",
    "Nevada": "US",
    "North Dakota": "US",
    "Oregon": "US",
    "South Dakota": "US",
    "Utah": "US",
    "Washington": "US",
    "Wisconsin": "US",
    "Wyoming": "US",
    "Arizona": "US",
    "Delaware": "US",
    "Kentucky": "US",
    "Maryland": "US",
    "New Jersey": "US",
    "North Carolina": "US",
    "Oklahoma": "US",
    "Pennsylvania": "US",
    "South Carolina": "US",
    "Texas": "US",
    "Virginia": "US",
    "New Mexico": "US",
    "New York": "US",
    "Alabama": "US",
    "Arkansas": "US",
    "Florida": "US",
    "Illinois": "US",
    "Indiana": "US",
    "Iowa": "US",
    "Louisiana": "US",
    "Mississippi": "US",
    "Missouri": "US",
    "Ohio": "US",
    "Tennessee": "US",
    "West Virginia": "US",
    "Maine": "US",
    "Massachusetts": "US",
    "Rhode Island": "US",
    "Andhra Pradesh": "IN",
    "Assam": "IN",
    "Gujarat": "IN",
    "Himachal Pradesh": "IN",
    "Karnataka": "IN",
    "Kerala": "IN",
    "Maharashtra": "IN",
    "Orissa": "IN",
    "Rajasthan": "IN",
    "Tamil Nadu": "IN",
    "West Bengal": "IN",
    "New South Wales": "AS",
    "Queensland": "AS",
    "South Australia": "AS",
    "Victoria": "AS",
    "Western Australia": "AS",
    "Altai Krai": "RS",
    "Bashkir ASSR": "RS",
    "Central and Western Ukraine": "UP",
    "Chelyabinsk": "RS",
    "Eastern and Southern Ukraine": "UP",
    "Kirovohrad": "RS",
    "Kursk": "RS",
    "Northeastern Ukraine": "UP",
    "Novosibirsk and Omsk": "",
    "Orenburg": "RS",
    "Rostov": "RS",
    "Saratov": "RS",
    "Southern Ukraine": "UP",
    "Stavropol Krai": "RS",
    "Voronezh": "RS",
}
for country in crop_calendar.Location.unique():
    if country in noaa_countries.ghcnd_country.tolist():
        uwm_to_noaa_country_map[country] = noaa_countries[noaa_countries.ghcnd_country==country].FIPS_country_code.values[0]

noaa_countries

,FIPS_country_code,ghcnd_country
0,AC,Antigua and Barbuda
1,AE,United Arab Emirates
2,AF,Afghanistan
3,AG,Algeria
4,AJ,Azerbaijan
5,AL,Albania
6,AM,Armenia
7,AO,Angola
8,AQ,American Samoa [United States]
9,AR,Argentina


In [324]:
# Clean up the crop yield data
# Remove rows which belong to dropped countries/regions
fao_crop_yield_data = fao_crop_yield_data[fao_crop_yield_data.fao_country_code.isin(country_data.fao_country_code)]

# Remove rows relating to unnecessary production_types.
# Yield = production/area_harvested so we can drop those two rows. 
print(fao_crop_yield_data.shape)
fao_crop_yield_data = fao_crop_yield_data[~fao_crop_yield_data.production_type.isin(['Area harvested', 'Production'])]
# Products with other production types are animals, which do not depend on weather.
# Find the corresponding product_ids and drop them
print(fao_crop_yield_data.shape)
product_types_to_drop = ['Stocks', 'Laying', 'Producing Animals/Slaughtered', 'Yield/Carcass Weight', 'Milk Animals', 'Prod Popultn']
products_to_drop = fao_crop_yield_data[fao_crop_yield_data.production_type.isin(production_types_to_drop)].product_id.unique()
fao_crop_yield_data = fao_crop_yield_data[~fao_crop_yield_data.product_id.isin(products_to_drop)]
print(fao_crop_yield_data.shape)

# Removing unnecessary columns
fao_crop_yield_data.drop(['fao_country_name', 'product', 'production_type_id', 'production_type'], axis=1, inplace=True, errors='ignore')

(1013013, 10)
(1013013, 10)
(482085, 10)


In [327]:
# Split the country_data to create two tables with primary keys
fao_country_codes = country_data[['fao_country_code', 'fao_country_name', 'noaa_country_code', 'EU']]
noaa_country_codes = country_data[['noaa_country_code', 'noaa_country_name', 'fao_country_code', 'EU']]
noaa_country_codes.head()

,noaa_country_code,noaa_country_name,fao_country_code,EU
35,AF,Afghanistan,2,False
36,AL,Albania,3,False
37,AG,Algeria,4,False
21,AQ,American Samoa [United States],244,False
38,AO,Angola,7,False


### Import FAO data aggregation codes

In [328]:
file_name = 'Production_Crops_Livestock_E_Flags'
fao_data_quality_description = pd.read_csv(f"{fao_file_path}fao_crop_data/normalized/{file_name}.csv", encoding='latin-1')
# Make column names lower case
fao_data_quality_description.columns = fao_data_quality_description.columns.str.lower()
fao_data_quality_description.rename({'flag': 'fao_data_quality_id'}, axis=1, inplace=True)
# Rename confusing data aggregation flags
fao_data_quality_description.replace('<blank>', 'O', inplace=True)
fao_data_quality_description.replace('*', 'U', inplace=True)
fao_data_quality_description

,fao_data_quality_id,description
0,U,Unofficial figure
1,O,Official data
2,A,Aggregate; may include official; semi-official...
3,F,FAO estimate
4,Fc,Calculated data
5,Im,FAO data based on imputation methodology
6,M,Data not available


### Import product item codes

In [329]:

file_name = 'Production_Crops_Livestock_E_ItemCodes'
fao_product_ids = pd.read_csv(f"{fao_file_path}fao_crop_data/normalized/{file_name}.csv", encoding='latin-1')
# Replace spaces column names with underscores and make lower case
fao_product_ids.columns = fao_product_ids.columns.str.replace(' ','_').str.lower()
# Rename columns so they aren't confused with the NOAA data
fao_product_ids.rename({'item': 'product', 'item_code': 'product_id'}, axis=1, inplace=True)
# Remove product_ids that don't exist in the fao_crop_yield data
fao_product_ids = fao_product_ids[fao_product_ids.product_id.isin(fao_crop_yield_data.product_id)]

fao_product_ids.head()

,product_id,cpc_code,product
0,101,'01195,Canary seed
9,103,'01199.02,Grain; mixed
23,108,'01199.90,Cereals nes
46,116,'01510,Potatoes
55,122,'01530,Sweet potatoes


In [289]:
yield_product_names = pd.merge(fao_crop_yield_data, fao_product_ids, left_on='product_id', right_on='product_id')
yield_product_names[yield_product_names['product']=='Hides; cattle; fresh']

,fao_country_code,product_id,year,unit,value,fao_data_quality_id,cpc_code,product
48663,2,919,1961,hg/An,200.0,Fc,'02951.01,Hides; cattle; fresh
48664,2,919,1962,hg/An,200.0,Fc,'02951.01,Hides; cattle; fresh
48665,2,919,1963,hg/An,200.0,Fc,'02951.01,Hides; cattle; fresh
48666,2,919,1964,hg/An,200.0,Fc,'02951.01,Hides; cattle; fresh
48667,2,919,1965,hg/An,200.0,Fc,'02951.01,Hides; cattle; fresh
...,...,...,...,...,...,...,...,...
58288,181,919,2016,hg/An,260.0,Fc,'02951.01,Hides; cattle; fresh
58289,181,919,2017,hg/An,261.0,Fc,'02951.01,Hides; cattle; fresh
58290,181,919,2018,hg/An,260.0,Fc,'02951.01,Hides; cattle; fresh
58291,181,919,2019,hg/An,260.0,Fc,'02951.01,Hides; cattle; fresh


## Push FAO crop yield data to database

In [330]:
class DatabaseInterface:
    def __init__(self,
                 db_name,
                 user,
                 password,
                 host='localhost',
                 port=3306,
                 driver='mysql+pymysql',
                 show_query_output=False):

        with create_engine(f'{driver}://{user}:{password}@{host}').connect() as db_connection:
            db_connection.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
        self.engine = create_engine(f"{driver}://{user}:{password}@{host}:{port}/{db_name}", echo=show_query_output)

    def insert_data(self, df: pd.DataFrame, table_name: str, dtypes: dict = {}, if_exists: str = 'replace', use_df_index: bool = False, chunk_size: int = 500):
        df.to_sql(
            name=table_name, 
            con=self.engine, 
            if_exists=if_exists, 
            index=use_df_index,
            chunksize=chunk_size,
            dtype=dtypes
        )

    def set_existing_field_as_primary_key(self, table_name: str, primary_key: str, constraints: str = ''):
        with self.engine.connect() as con:
            con.execute(f"ALTER TABLE `{table_name}` ADD PRIMARY KEY (`{primary_key}`) {constraints};")

    def add_new_field_as_primary_key(self, table_name: str, primary_key: str, primary_key_dtype: str, constraints: str = ''):
        with self.engine.connect() as con:
            con.execute(f"ALTER TABLE `{table_name}` ADD `{primary_key}` {primary_key_dtype} PRIMARY KEY {constraints};")

    def set_foreign_key(self, table_name: str, foreign_key: str, foreign_table_name: str, foreign_table_key:str):
        with self.engine.connect() as con:
            con.execute(f"ALTER TABLE `{table_name}` ADD FOREIGN KEY (`{foreign_key}`) REFERENCES `{foreign_table_name}`(`{foreign_table_key}`);")

    def close_connection(self):
        self.engine.dispose()

In [331]:
dbi = DatabaseInterface(db_name='crop_yield_prediction', 
                        user=local_mysql_user, 
                        password=local_mysql_password)

dbi.engine.execute(f"SET FOREIGN_KEY_CHECKS=0;")
dbi.engine.execute(f"DROP TABLE IF EXISTS fao_data_quality;")
dbi.engine.execute(f"DROP TABLE IF EXISTS fao_products;")
dbi.engine.execute(f"DROP TABLE IF EXISTS fao_countries;")
dbi.engine.execute(f"DROP TABLE IF EXISTS noaa_countries;")
dbi.engine.execute(f"DROP TABLE IF EXISTS fao_crop_yields;")
dbi.engine.execute(f"SET FOREIGN_KEY_CHECKS=1;")

print("Inserting fao_data_quality")
dbi.insert_data(
   df=fao_data_quality_description, 
   table_name='fao_data_quality', 
   dtypes={
      'fao_data_quality_id': String(2),
      'description': Text,
      }
)
dbi.set_existing_field_as_primary_key('fao_data_quality', 'fao_data_quality_id')

print("Inserting fao_products")
dbi.insert_data(
   df=fao_product_ids, 
   table_name='fao_products', 
   dtypes={
      'product_id': Integer,
      'cpc_code': String(20),
      'product': Text,
      }
)
dbi.set_existing_field_as_primary_key('fao_products', 'product_id')

print("Inserting fao_countries")
dbi.insert_data(
   df=fao_country_codes, 
   table_name='fao_countries', 
   dtypes={
      'fao_country_code': Integer,
      'fao_country_name': Text,
      'noaa_country_code': String(2),
      'EU': Boolean,
      }
)
dbi.set_existing_field_as_primary_key('fao_countries', 'fao_country_code')

# Uncomment this line if noaa_weather_data table has already been created
dbi.engine.execute(f"ALTER TABLE noaa_weather_data DROP FOREIGN KEY noaa_weather_data_ibfk_1;")
print("Inserting noaa_countries")
dbi.insert_data(
   df=noaa_country_codes, 
   table_name='noaa_countries', 
   dtypes={
      'noaa_country_code': String(2),
      'noaa_country_name': Text,
      'fao_country_code': Integer,
      'EU': Boolean,
      }
)
dbi.set_existing_field_as_primary_key('noaa_countries', 'noaa_country_code')

print("Inserting fao_crop_yields")
dbi.insert_data(
   df=fao_crop_yield_data, 
   table_name='fao_crop_yields', 
   dtypes={
      'id': Integer,
      'fao_country_code': Integer,
      'product_id': Integer,
      'year': Integer,
      'unit': Text,
      'value': Integer,
      'fao_data_quality_id': String(2),
      }
)
dbi.add_new_field_as_primary_key(table_name='fao_crop_yields', primary_key='id', primary_key_dtype='INT', constraints='AUTO_INCREMENT')

print('Setting foreign keys')
dbi.set_foreign_key(table_name='fao_countries', foreign_key='noaa_country_code', foreign_table_name='noaa_countries', foreign_table_key='noaa_country_code')
dbi.set_foreign_key(table_name='noaa_countries', foreign_key='fao_country_code', foreign_table_name='fao_countries', foreign_table_key='fao_country_code')
dbi.set_foreign_key(table_name='fao_crop_yields', foreign_key='fao_country_code', foreign_table_name='fao_countries', foreign_table_key='fao_country_code')
dbi.set_foreign_key(table_name='fao_crop_yields', foreign_key='product_id', foreign_table_name='fao_products', foreign_table_key='product_id')
dbi.set_foreign_key(table_name='fao_crop_yields', foreign_key='fao_data_quality_id', foreign_table_name='fao_data_quality', foreign_table_key='fao_data_quality_id')

# Uncomment this line if noaa_weather_data table has already been created
dbi.set_foreign_key(table_name='noaa_weather_data', foreign_key='noaa_country_code', foreign_table_name='noaa_countries', foreign_table_key='noaa_country_code')
print("Closing connection")
dbi.close_connection()
print("Done")



Inserting fao_data_quality
Inserting fao_products
Inserting fao_countries
Inserting noaa_countries
Inserting fao_crop_yields
Setting foreign keys
Closing connection
Done


# Data exploration and cleaning 
## NOAA Global Historical Climatology Network Daily (GHCN-D) from CSV file
AWS bucket url: https://noaa-ghcn-pds.s3.amazonaws.com/index.html#csv/

## Create dataframes for datatype descriptions

In [38]:
soil_temp_units = 'tenths of degrees C'
# Minimum soil temperature element_id=SN*#, where * corresponds to a code for ground cover and # corresponds to a code for soil depth.
# Maximum soil temperature element_id=SX*#, where * corresponds to a code for ground cover and # corresponds to a code for soil depth.

ground_cover_map = { 
    '0': "unknown",
    '1': "grass",
    '2': "fallow",
    '3': "bare ground",
    '4': "brome grass",
    '5': "sod",
    '6': "straw mulch",
    '7': "grass muck",
    '8': "bare muck",
}
soil_depth_map = {
    '1': "5 cm",
    '2': "10 cm",
    '3': "20 cm",
    '4': "50 cm",
    '5': "100 cm",
    '6': "150 cm",
    '7': "180 cm",
}

min_soil_temp_data_type_ids = []
max_soil_temp_data_type_ids = []

for gc_id, gc_desc in ground_cover_map.items():
    for sd_id, sd_desc in soil_depth_map.items():
        min_soil_temp_data_type_ids.append({
            'data_type_id': 'SN'+gc_id+sd_id, 
            'description': f'Minimum soil temperature: Ground cover = {gc_desc}, Soil depth = {sd_desc}',
            'units': soil_temp_units
            })
        max_soil_temp_data_type_ids.append({
            'data_type_id': 'SX'+gc_id+sd_id, 
            'description': f'Maximum soil temperature: Ground cover = {gc_desc}, Soil depth = {sd_desc}',
            'units': soil_temp_units
            })
pd.DataFrame(min_soil_temp_data_type_ids).head()


,data_type_id,description,units
0,SN01,Minimum soil temperature: Ground cover = unkno...,tenths of degrees C
1,SN02,Minimum soil temperature: Ground cover = unkno...,tenths of degrees C
2,SN03,Minimum soil temperature: Ground cover = unkno...,tenths of degrees C
3,SN04,Minimum soil temperature: Ground cover = unkno...,tenths of degrees C
4,SN05,Minimum soil temperature: Ground cover = unkno...,tenths of degrees C


In [39]:
noaa_data_type_ids = [
    {'data_type_id': 'PRCP', 'description': 'Precipitation', 'units': 'tenths of mm'},
    {'data_type_id': 'SNOW', 'description': 'Snowfall', 'units': 'mm'},
    {'data_type_id': 'SNWD', 'description': 'Snow depth', 'units': 'mm'},
    {'data_type_id': 'TMAX', 'description': 'Maximum temperature', 'units': 'tenths of degrees C'},
    {'data_type_id': 'TMIN', 'description': 'Minimum temperature', 'units': 'tenths of degrees C'},
    {'data_type_id': 'ACMC', 'description': 'Average cloudiness midnight to midnight from 30-second ceilometer data', 'units': 'percent'},
    {'data_type_id': 'ACMH', 'description': 'Average cloudiness midnight to midnight from manual observations', 'units': 'percent'},
    {'data_type_id': 'ACSC', 'description': 'Average cloudiness sunrise to sunset from 30-second ceilometer data', 'units': 'percent'},
    {'data_type_id': 'ACSH', 'description': 'Average cloudiness sunrise to sunset from manual observations', 'units': 'percent'},
    {'data_type_id': 'AWDR', 'description': 'Average daily wind direction', 'units': 'degrees'},
    {'data_type_id': 'AWND', 'description': 'Average daily wind speed', 'units': 'tenths of meters per second'},
    {'data_type_id': 'EVAP', 'description': 'Evaporation of water from evaporation pan', 'units': 'tenths of mm'},
    {'data_type_id': 'FRGB', 'description': 'Base of frozen ground layer', 'units': 'cm'},
    {'data_type_id': 'FRGT', 'description': 'Top of frozen ground layer', 'units': 'cm'},
    {'data_type_id': 'FRTH', 'description': 'Thickness of frozen ground layer', 'units': 'cm'},
    {'data_type_id': 'GAHT', 'description': 'Difference between river and gauge height', 'units': 'cm'},
    {'data_type_id': 'MNPN', 'description': 'Daily minimum temperature of water in an evaporation pan', 'units': 'tenths of degrees C'},
    {'data_type_id': 'MXPN', 'description': 'Daily maximum temperature of water in an evaporation pan', 'units': 'tenths of degrees C'},
    {'data_type_id': 'PGTM', 'description': 'Peak gust time', 'units': '(hours and minutes, i.e., HHMM)'},
    {'data_type_id': 'PSUN', 'description': 'Daily percent of possible sunshine', 'units': 'percent'},
    {'data_type_id': 'THIC', 'description': 'Thickness of ice on water', 'units': 'tenths of mm'},
    {'data_type_id': 'TOBS', 'description': 'Temperature at the time of observation', 'units': 'tenths of degrees C'},
    {'data_type_id': 'TSUN', 'description': 'Daily total sunshine', 'units': 'minutes'},
    {'data_type_id': 'WDF1', 'description': 'Direction of fastest 1-minute wind', 'units': 'degrees'},
    {'data_type_id': 'WDF2', 'description': 'Direction of fastest 2-minute wind', 'units': 'degrees'},
    {'data_type_id': 'WDF5', 'description': 'Direction of fastest 5-second wind', 'units': 'degrees'},
    {'data_type_id': 'WDFG', 'description': 'Direction of peak wind gust', 'units': 'degrees'},
    {'data_type_id': 'WDFI', 'description': 'Direction of highest instantaneous wind', 'units': 'degrees'},
    {'data_type_id': 'WDFM', 'description': 'Fastest mile wind direction', 'units': 'degrees'},
    {'data_type_id': 'WDMV', 'description': '24-hour wind movement', 'units': 'km'},
    {'data_type_id': 'WESD', 'description': 'Water equivalent of snow on the ground', 'units': 'tenths of mm'},
    {'data_type_id': 'WESF', 'description': 'Water equivalent of snowfall', 'units': 'tenths of mm'},
    {'data_type_id': 'WSF1', 'description': 'Fastest 1-minute wind speed', 'units': 'tenths of meters per second'},
    {'data_type_id': 'WSF2', 'description': 'Fastest 2-minute wind speed', 'units': 'tenths of meters per second'},
    {'data_type_id': 'WSF5', 'description': 'Fastest 5-second wind speed', 'units': 'tenths of meters per second'},
    {'data_type_id': 'WSFG', 'description': 'Peak gust wind speed', 'units': 'tenths of meters per second'},
    {'data_type_id': 'WSFI', 'description': 'Highest instantaneous wind speed', 'units': 'tenths of meters per second'},
    {'data_type_id': 'WSFM', 'description': 'Fastest mile wind speed', 'units': 'tenths of meters per second'},
    {'data_type_id': 'TAVG', 'description': "Average temperature", "units": "tenths of degrees C"}, # [Note that TAVG from source 'S' corresponds to an average for the period ending at 2400 UTC rather than local midnight]
    {'data_type_id': 'WT01', 'description': "Weather type: Fog, ice fog, or freezing fog (may include heavy fog)", 'units': 'Boolean'},
    {'data_type_id': 'WT02', 'description': "Weather type: Heavy fog or heaving freezing fog (not always distinguished from fog)", 'units': 'Boolean'},
    {'data_type_id': 'WT03', 'description': "Weather type: Thunder", 'units': 'Boolean'},
    {'data_type_id': 'WT04', 'description': "Weather type: Ice pellets, sleet, snow pellets, or small hail", 'units': 'Boolean'},
    {'data_type_id': 'WT05', 'description': "Weather type: Hail (may include small hail)", 'units': 'Boolean'},
    {'data_type_id': 'WT06', 'description': "Weather type: Glaze or rime", 'units': 'Boolean'},
    {'data_type_id': 'WT07', 'description': "Weather type: Dust, volcanic ash, blowing dust, blowing sand, or blowing obstruction", 'units': 'Boolean'},
    {'data_type_id': 'WT08', 'description': "Weather type: Smoke or haze", 'units': 'Boolean'},
    {'data_type_id': 'WT09', 'description': "Weather type: Blowing or drifting snow", 'units': 'Boolean'},
    {'data_type_id': 'WT10', 'description': "Weather type: Tornado, waterspout, or funnel cloud", 'units': 'Boolean'},
    {'data_type_id': 'WT11', 'description': "Weather type: High or damaging winds", 'units': 'Boolean'},
    {'data_type_id': 'WT12', 'description': "Weather type: Blowing spray", 'units': 'Boolean'},
    {'data_type_id': 'WT13', 'description': "Weather type: Mist", 'units': 'Boolean'},
    {'data_type_id': 'WT14', 'description': "Weather type: Drizzle", 'units': 'Boolean'},
    {'data_type_id': 'WT15', 'description': "Weather type: Freezing drizzle", 'units': 'Boolean'},
    {'data_type_id': 'WT16', 'description': "Weather type: Rain (may include freezing rain, drizzle, and freezing drizzle)", 'units': 'Boolean'},
    {'data_type_id': 'WT17', 'description': "Weather type: Freezing rain", 'units': 'Boolean'},
    {'data_type_id': 'WT18', 'description': "Weather type: Snow, snow pellets, snow grains, or ice crystals", 'units': 'Boolean'},
    {'data_type_id': 'WT19', 'description': "Weather type: Unknown source of precipitation", 'units': 'Boolean'},
    {'data_type_id': 'WT21', 'description': "Weather type: Ground fog", 'units': 'Boolean'},
    {'data_type_id': 'WT22', 'description': "Weather type: Ice fog or freezing fog", 'units': 'Boolean'},
    {'data_type_id': 'WV01', 'description': "Weather in vicinity: Fog, ice fog, or freezing fog (may include heavy fog)", 'units': 'Boolean'},
    {'data_type_id': 'WV03', 'description': "Weather in vicinity: Thunder", 'units': 'Boolean'},
    {'data_type_id': 'WV07', 'description': "Weather in vicinity: Ash, dust, sand, or other blowing obstruction", 'units': 'Boolean'},
    {'data_type_id': 'WV18', 'description': "Weather in vicinity: Snow or ice crystals", 'units': 'Boolean'},
    {'data_type_id': 'WV20', 'description': "Weather in vicinity: Rain or snow shower", 'units': 'Boolean'},
    {'data_type_id': 'FMTM', 'description': 'Time of fastest mile or fastest 1-minute wind', 'units': 'hours and minutes,i.e., HHMM'},
    {'data_type_id': 'DASF', 'description': 'Number of days included in the multiday snowfall total', 'units': 'MDSF'},
    {'data_type_id': 'MDSF', 'description': 'Multiday snowfall total', 'units': 'mm'},
    {'data_type_id': 'DAWM', 'description': 'Number of days included in the multiday wind movement', 'units': 'MDWM'},
    {'data_type_id': 'MDWM', 'description': 'Multiday wind movement', 'units': 'km'},
    {'data_type_id': 'DAEV', 'description': 'Number of days included in the multiday evaporation total', 'units': 'MDEV'},
    {'data_type_id': 'MDEV', 'description': 'Multiday evaporation total; (use with DAEV)', 'units': 'tenths of mm'},
    {'data_type_id': 'DWPR', 'description': 'Number of days with non-zero precipitation included in multiday precipitation total', 'units': 'MDPR'},
    {'data_type_id': 'DAPR', 'description': 'Number of days included in the multiday precipitation total', 'units': 'MDPR'},
    {'data_type_id': 'MDPR', 'description': 'Multiday precipitation total; (use with DAPR and DWPR, if available)', 'units': 'tenths of mm'},
    {'data_type_id': 'DATN', 'description': 'Number of days included in the multiday minimum temperature', 'units': 'MDTN'},
    {'data_type_id': 'MDTN', 'description': 'Multiday minimum temperature; (use with DATN)', 'units': 'tenths of degrees C'},
    {'data_type_id': 'DATX', 'description': 'Number of days included in the multiday maximum temperature', 'units': 'MDTX'},
    {'data_type_id': 'MDTX', 'description': 'Multiday maximum temperature; (use with DATX)', 'units': 'tenths of degrees C'},
]
noaa_data_type_ids.extend(max_soil_temp_data_type_ids)
noaa_data_type_ids.extend(min_soil_temp_data_type_ids)
noaa_data_type_ids = pd.DataFrame(noaa_data_type_ids)
noaa_data_type_ids.head()

,data_type_id,description,units
0,PRCP,Precipitation,tenths of mm
1,SNOW,Snowfall,mm
2,SNWD,Snow depth,mm
3,TMAX,Maximum temperature,tenths of degrees C
4,TMIN,Minimum temperature,tenths of degrees C


### Select data types to keep and to drop

In [202]:
data_types_to_keep = [
    {'data_type_id': 'PRCP', 'description': 'Precipitation', 'units': 'tenths of mm'},
    {'data_type_id': 'SNOW', 'description': 'Snowfall', 'units': 'mm'},
    {'data_type_id': 'SNWD', 'description': 'Snow depth', 'units': 'mm'},
    {'data_type_id': 'TMAX', 'description': 'Maximum temperature', 'units': 'tenths of degrees C'},
    {'data_type_id': 'TMIN', 'description': 'Minimum temperature', 'units': 'tenths of degrees C'},
    {'data_type_id': 'ACMH', 'description': 'Average cloudiness midnight to midnight from manual observations', 'units': 'percent'},
    {'data_type_id': 'AWND', 'description': 'Average daily wind speed', 'units': 'tenths of meters per second'},
    {'data_type_id': 'EVAP', 'description': 'Evaporation of water from evaporation pan', 'units': 'tenths of mm'},
    {'data_type_id': 'FRTH', 'description': 'Thickness of frozen ground layer', 'units': 'cm'},
    {'data_type_id': 'PSUN', 'description': 'Daily percent of possible sunshine', 'units': 'percent'},
    {'data_type_id': 'THIC', 'description': 'Thickness of ice on water', 'units': 'tenths of mm'},
    {'data_type_id': 'TOBS', 'description': 'Temperature at the time of observation', 'units': 'tenths of degrees C'},
    {'data_type_id': 'TSUN', 'description': 'Daily total sunshine', 'units': 'minutes'},
    {'data_type_id': 'WDMV', 'description': '24-hour wind movement', 'units': 'km'},
    {'data_type_id': 'TAVG', 'description': "Average temperature', 'units': 'tenths of degrees C"},
    {'data_type_id': 'WT01', 'description': "Weather type: Fog, ice fog, or freezing fog (may include heavy fog)", 'units': 'Boolean'},
    {'data_type_id': 'WT02', 'description': "Weather type: Heavy fog or heaving freezing fog (not always distinguished from fog)", 'units': 'Boolean'},
    {'data_type_id': 'WT03', 'description': "Weather type: Thunder", 'units': 'Boolean'},
    {'data_type_id': 'WT04', 'description': "Weather type: Ice pellets, sleet, snow pellets, or small hail", 'units': 'Boolean'},
    {'data_type_id': 'WT05', 'description': "Weather type: Hail (may include small hail)", 'units': 'Boolean'},
    {'data_type_id': 'WT06', 'description': "Weather type: Glaze or rime", 'units': 'Boolean'},
    {'data_type_id': 'WT07', 'description': "Weather type: Dust, volcanic ash, blowing dust, blowing sand, or blowing obstruction", 'units': 'Boolean'},
    {'data_type_id': 'WT08', 'description': "Weather type: Smoke or haze", 'units': 'Boolean'},
    {'data_type_id': 'WT09', 'description': "Weather type: Blowing or drifting snow", 'units': 'Boolean'},
    {'data_type_id': 'WT10', 'description': "Weather type: Tornado, waterspout, or funnel cloud", 'units': 'Boolean'},
    {'data_type_id': 'WT11', 'description': "Weather type: High or damaging winds", 'units': 'Boolean'},
    {'data_type_id': 'WT12', 'description': "Weather type: Blowing spray", 'units': 'Boolean'},
    {'data_type_id': 'WT13', 'description': "Weather type: Mist", 'units': 'Boolean'},
    {'data_type_id': 'WT14', 'description': "Weather type: Drizzle", 'units': 'Boolean'},
    {'data_type_id': 'WT15', 'description': "Weather type: Freezing drizzle", 'units': 'Boolean'},
    {'data_type_id': 'WT16', 'description': "Weather type: Rain (may include freezing rain, drizzle, and freezing drizzle)", 'units': 'Boolean'},
    {'data_type_id': 'WT17', 'description': "Weather type: Freezing rain", 'units': 'Boolean'},
    {'data_type_id': 'WT18', 'description': "Weather type: Snow, snow pellets, snow grains, or ice crystals", 'units': 'Boolean'},
    {'data_type_id': 'WT19', 'description': "Weather type: Unknown source of precipitation", 'units': 'Boolean'},
    {'data_type_id': 'WT21', 'description': "Weather type: Ground fog", 'units': 'Boolean'},
    {'data_type_id': 'WT22', 'description': "Weather type: Ice fog or freezing fog", 'units': 'Boolean'},
]

data_types_to_drop = [
    {'data_type_id': 'AWDR', 'description': 'Average daily wind direction', 'units': 'degrees'},
    {'data_type_id': 'GAHT', 'description': 'Difference between river and gauge height', 'units': 'cm'},
    {'data_type_id': 'MNPN', 'description': 'Daily minimum temperature of water in an evaporation pan', 'units': 'tenths of degrees C'},
    {'data_type_id': 'MXPN', 'description': 'Daily maximum temperature of water in an evaporation pan', 'units': 'tenths of degrees C'},
    {'data_type_id': 'WESD', 'description': 'Water equivalent of snow on the ground', 'units': 'tenths of mm'},
    {'data_type_id': 'WESF', 'description': 'Water equivalent of snowfall', 'units': 'tenths of mm'},
    {'data_type_id': 'WSFM', 'description': 'Fastest mile wind speed', 'units': 'tenths of meters per second'},
    {'data_type_id': 'FRGB', 'description': 'Base of frozen ground layer', 'units': 'cm'},
    {'data_type_id': 'FRGT', 'description': 'Top of frozen ground layer', 'units': 'cm'},
    {'data_type_id': 'ACMC', 'description': 'Average cloudiness midnight to midnight from 30-second ceilometer data', 'units': 'percent'},
    {'data_type_id': 'ACSC', 'description': 'Average cloudiness sunrise to sunset from 30-second ceilometer data', 'units': 'percent'},
    {'data_type_id': 'ACSH', 'description': 'Average cloudiness sunrise to sunset from manual observations', 'units': 'percent'},
    {'data_type_id': 'PGTM', 'description': 'Peak gust time', 'units': '(hours and minutes, i.e., HHMM)'},
    {'data_type_id': 'WDFG', 'description': 'Direction of peak wind gust', 'units': 'degrees'},
    {'data_type_id': 'WDFI', 'description': 'Direction of highest instantaneous wind', 'units': 'degrees'},
    {'data_type_id': 'WDFM', 'description': 'Fastest mile wind direction', 'units': 'degrees'},
    {'data_type_id': 'WSF1', 'description': 'Fastest 1-minute wind speed', 'units': 'tenths of meters per second'},
    {'data_type_id': 'WSF2', 'description': 'Fastest 2-minute wind speed', 'units': 'tenths of meters per second'},
    {'data_type_id': 'WSF5', 'description': 'Fastest 5-second wind speed', 'units': 'tenths of meters per second'},
    {'data_type_id': 'WSFG', 'description': 'Peak gust wind speed', 'units': 'tenths of meters per second'},
    {'data_type_id': 'WSFI', 'description': 'Highest instantaneous wind speed', 'units': 'tenths of meters per second'},
    {'data_type_id': 'WDF1', 'description': 'Direction of fastest 1-minute wind', 'units': 'degrees'},
    {'data_type_id': 'WDF2', 'description': 'Direction of fastest 2-minute wind', 'units': 'degrees'},
    {'data_type_id': 'WDF5', 'description': 'Direction of fastest 5-second wind', 'units': 'degrees'},
    {'data_type_id': 'WV01', 'description': "Weather in vicinity: Fog, ice fog, or freezing fog (may include heavy fog)", 'units': 'Boolean'},
    {'data_type_id': 'WV03', 'description': "Weather in vicinity: Thunder", 'units': 'Boolean'},
    {'data_type_id': 'WV07', 'description': "Weather in vicinity: Ash, dust, sand, or other blowing obstruction", 'units': 'Boolean'},
    {'data_type_id': 'WV18', 'description': "Weather in vicinity: Snow or ice crystals", 'units': 'Boolean'},
    {'data_type_id': 'WV20', 'description': "Weather in vicinity: Rain or snow shower", 'units': 'Boolean'},
    {'data_type_id': 'FMTM', 'description': 'Time of fastest mile or fastest 1-minute wind', 'units': 'hours and minutes,i.e., HHMM'},
    {'data_type_id': 'DASF', 'description': 'Number of days included in the multiday snowfall total', 'units': 'MDSF'},
    {'data_type_id': 'MDSF', 'description': 'Multiday snowfall total', 'units': 'mm'},
    {'data_type_id': 'DAWM', 'description': 'Number of days included in the multiday wind movement', 'units': 'MDWM'},
    {'data_type_id': 'MDWM', 'description': 'Multiday wind movement', 'units': 'km'},
    {'data_type_id': 'DAEV', 'description': 'Number of days included in the multiday evaporation total', 'units': 'MDEV'},
    {'data_type_id': 'MDEV', 'description': 'Multiday evaporation total; (use with DAEV)', 'units': 'tenths of mm'},
    {'data_type_id': 'DWPR', 'description': 'Number of days with non-zero precipitation included in multiday precipitation total', 'units': 'MDPR'},
    {'data_type_id': 'DAPR', 'description': 'Number of days included in the multiday precipitation total', 'units': 'MDPR'},
    {'data_type_id': 'MDPR', 'description': 'Multiday precipitation total; (use with DAPR and DWPR, if available)', 'units': 'tenths of mm'},
    {'data_type_id': 'DATN', 'description': 'Number of days included in the multiday minimum temperature', 'units': 'MDTN'},
    {'data_type_id': 'MDTN', 'description': 'Multiday minimum temperature; (use with DATN)', 'units': 'tenths of degrees C'},
    {'data_type_id': 'DATX', 'description': 'Number of days included in the multiday maximum temperature', 'units': 'MDTX'},
    {'data_type_id': 'MDTX', 'description': 'Multiday maximum temperature; (use with DATX)', 'units': 'tenths of degrees C'},
]
data_ids_to_keep = [data_type['data_type_id'] for data_type in data_types_to_keep]
noaa_data_type_ids['use_in_model'] = noaa_data_type_ids.data_type_id.apply(lambda x: True if x in data_ids_to_keep else False)
noaa_data_type_ids.head()

,data_type_id,description,units,use_in_model
0,PRCP,Precipitation,tenths of mm,True
1,SNOW,Snowfall,mm,True
2,SNWD,Snow depth,mm,True
3,TMAX,Maximum temperature,tenths of degrees C,True
4,TMIN,Minimum temperature,tenths of degrees C,True


In [204]:
dbi = DatabaseInterface(db_name='crop_yield_prediction', 
                        user=local_mysql_user, 
                        password=local_mysql_password)

dbi.engine.execute(f"DROP TABLE IF EXISTS noaa_data_types;")

print("Inserting noaa_data_types")
dbi.insert_data(
   df=noaa_data_type_ids, 
   table_name='noaa_data_types', 
   dtypes={
      'data_type_id': String(4),
      'description': Text,
      'units': Text,
      'use_in_model': Boolean,
      }
)
dbi.set_existing_field_as_primary_key('noaa_data_types', 'data_type_id')

print("Closing connection")
dbi.close_connection()
print("Done")

Inserting noaa_data_types
Closing connection
Done


## Import the NOAA data csv

In [222]:
custom_date_parser = lambda x: datetime.strptime(x, "%Y%m%d")

noaa_ghcnd_file_path = './data/noaa_ghcn_aws_data/'

file_name = '2021'
noaa_ghcn_aws_data_2021 = pd.read_csv(
    f"{noaa_ghcnd_file_path}{file_name}.csv", 
    header=None, 
    usecols=[0,1,2,3],
    names=['station_id', 'date', 'data_type_id', 'data_value'],#, 'measurement_id', 'quality_id', 'source_id', 'observation_time'],
    nrows=100000, 
    parse_dates=['date'],
    date_parser=custom_date_parser,
    # dtype={'observation_time':str}
    )
#noaa_ghcn_aws_data_2021.observation_time.replace({np.nan: '0000', '2400': '0000'}, inplace=True)
#noaa_ghcn_aws_data_2021.observation_time = pd.to_datetime(noaa_ghcn_aws_data_2021.observation_time, format='%H%M').dt.time

# Extract country code from station_id
noaa_ghcn_aws_data_2021['noaa_country_code'] = noaa_ghcn_aws_data_2021.station_id.str[:2]
# Drop countries that are not in the fao dataset
noaa_ghcn_aws_data_2021 = noaa_ghcn_aws_data_2021[noaa_ghcn_aws_data_2021.noaa_country_code.isin(noaa_country_codes.noaa_country_code)]
# Drop unnecessary data types
noaa_ghcn_aws_data_2021 = noaa_ghcn_aws_data_2021[noaa_ghcn_aws_data_2021.data_type_id.isin(data_ids_to_keep)]
# Pivot the table.
noaa_weather_data = noaa_ghcn_aws_data_2021.pivot(index=['noaa_country_code', 'station_id', 'date'], columns='data_type_id', values='data_value')
# Fill in the weather type columns with zeroes because they are one-hot encoded/binary
# weather_type_cols = noaa_weather_data.columns[noaa_weather_data.columns.str.contains('WT')]
# noaa_weather_data[weather_type_cols] = noaa_weather_data[weather_type_cols].fillna(0)

noaa_weather_data.reset_index(inplace=True)
noaa_weather_data.head()

data_type_id,noaa_country_code,station_id,date,AWND,EVAP,PRCP,PSUN,SNOW,SNWD,TAVG,THIC,TMAX,TMIN,TOBS,TSUN,WDMV,WT01,WT02,WT03,WT04,WT05,WT06,WT08,WT09,WT11
0,AE,AE000041196,2021-01-01,NaN,NaN,0.0,NaN,NaN,NaN,214.0,NaN,278.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AE,AE000041196,2021-01-02,NaN,NaN,0.0,NaN,NaN,NaN,211.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AE,AEM00041194,2021-01-01,NaN,NaN,0.0,NaN,NaN,NaN,217.0,NaN,266.0,178.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,AE,AEM00041194,2021-01-02,NaN,NaN,0.0,NaN,NaN,NaN,214.0,NaN,262.0,166.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AE,AEM00041217,2021-01-01,NaN,NaN,NaN,NaN,NaN,NaN,202.0,NaN,262.0,155.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37034,WA,WA007848390,2021-01-01,NaN,NaN,NaN,NaN,NaN,NaN,220.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37035,WA,WA007878380,2021-01-01,NaN,NaN,79.0,NaN,NaN,NaN,209.0,NaN,346.0,189.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37036,WA,WA010517310,2021-01-01,NaN,NaN,NaN,NaN,NaN,NaN,244.0,NaN,332.0,200.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37037,WA,WAM00068006,2021-01-01,NaN,NaN,NaN,NaN,NaN,NaN,283.0,NaN,326.0,199.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [154]:
noaa_weather_data.shape

(343038, 25)

In [250]:
[data_type['data_type_id'] for datatype in data_types_to_keep]

NameError: name 'data_type' is not defined

#### Insert data sample to test values

In [251]:
custom_date_parser = lambda x: datetime.strptime(x, "%Y%m%d")

noaa_ghcnd_file_path = './data/noaa_ghcn_aws_data/'

file_name = '2021'
noaa_ghcn_aws_data_2021 = pd.read_csv(
    f"{noaa_ghcnd_file_path}{file_name}.csv", 
    header=None, 
    usecols=[0,1,2,3],
    names=['station_id', 'date', 'data_type_id', 'data_value'],#, 'measurement_id', 'quality_id', 'source_id', 'observation_time'],
    nrows=100000, 
    parse_dates=['date'],
    date_parser=custom_date_parser,
    )
# Extract country code from station_id
noaa_ghcn_aws_data_2021['noaa_country_code'] = noaa_ghcn_aws_data_2021.station_id.str[:2]
# Drop countries that are not in the fao dataset
noaa_ghcn_aws_data_2021 = noaa_ghcn_aws_data_2021[noaa_ghcn_aws_data_2021.noaa_country_code.isin(noaa_country_codes.noaa_country_code)]

# Pivot the table.
noaa_weather_data = noaa_ghcn_aws_data_2021.pivot(index=['noaa_country_code', 'station_id', 'date'], columns='data_type_id', values='data_value')
# Fill in the weather type columns with zeroes because they are one-hot encoded/binary
weather_type_cols = noaa_weather_data.columns[noaa_weather_data.columns.str.contains('WT')]
noaa_weather_data[weather_type_cols] = noaa_weather_data[weather_type_cols].fillna(0)

noaa_weather_data.reset_index(inplace=True)

# Create the table with a predefined autoincrementing primary key so further data can be added
dbi = DatabaseInterface(db_name='crop_yield_prediction', 
                        user=local_mysql_user, 
                        password=local_mysql_password)

print("Inserting noaa_weather_data")
dbi.insert_data(noaa_weather_data, 'test_data', if_exists='append')

print("Closing connection")
dbi.close_connection()
print("Done")


Inserting noaa_weather_data
Closing connection
Done


### Insert the NOAA weather data

In [248]:
# Create the table with a predefined autoincrementing primary key so further data can be added
dbi = DatabaseInterface(db_name='crop_yield_prediction', 
                        user=local_mysql_user, 
                        password=local_mysql_password)

dbi.engine.execute(f"DROP TABLE IF EXISTS noaa_weather_data;")

meta = MetaData()
meta.bind = dbi.engine
meta.reflect()

noaa_weather_data_table = Table(
   'noaa_weather_data', meta, 
   Column('id', Integer, primary_key=True), 
   Column('noaa_country_code', String(2), ForeignKey("noaa_countries.noaa_country_code")), 
   Column('station_id', Text),
   Column('date', Date),
   Column('PRCP', Float(50)),
   Column('SNOW', Float(50)),
   Column('SNWD', Float(50)),
   Column('TMAX', Float(50)),
   Column('TMIN', Float(50)),
   Column('ACMH', Float(50)),
   Column('AWND', Float(50)),
   Column('EVAP', Float(50)),
   Column('FRTH', Float(50)),
   Column('PSUN', Float(50)),
   Column('THIC', Float(50)),
   Column('TOBS', Float(50)),
   Column('TSUN', Float(50)),
   Column('WDMV', Float(50)),
   Column('TAVG', Float(50)),
   Column('WT01', Boolean),
   Column('WT02', Boolean),
   Column('WT03', Boolean),
   Column('WT04', Boolean),
   Column('WT05', Boolean),
   Column('WT06', Boolean),
   Column('WT07', Boolean),
   Column('WT08', Boolean),
   Column('WT09', Boolean),
   Column('WT10', Boolean),
   Column('WT11', Boolean),
   Column('WT12', Boolean),
   Column('WT13', Boolean),
   Column('WT14', Boolean),
   Column('WT15', Boolean),
   Column('WT16', Boolean),
   Column('WT17', Boolean),
   Column('WT18', Boolean),
   Column('WT19', Boolean),
   Column('WT21', Boolean),
   Column('WT22', Boolean)
)
                                  
noaa_weather_data_table.create(dbi.engine)
# meta.create_all(dbi.engine)

print("Inserting noaa_weather_data")
dbi.insert_data(noaa_weather_data, 'noaa_weather_data', if_exists='append')

print("Closing connection")
dbi.close_connection()
print("Done")

Inserting noaa_weather_data
Closing connection
Done


## Combine the process of importing the CSV and exporting to the database and split the process to handle chunks of data

In [249]:
# Create the table with a predefined autoincrementing primary key so further data can be added
dbi = DatabaseInterface(db_name='crop_yield_prediction', 
                        user=local_mysql_user, 
                        password=local_mysql_password)

dbi.engine.execute(f"DROP TABLE IF EXISTS noaa_weather_data;")

meta = MetaData()
meta.bind = dbi.engine
meta.reflect()

noaa_weather_data_table = Table(
   'noaa_weather_data', meta, 
   Column('id', Integer, primary_key=True), 
   Column('noaa_country_code', String(2), ForeignKey("noaa_countries.noaa_country_code")), 
   Column('station_id', Text),
   Column('date', Date),
   Column('PRCP', Float(50)),
   Column('SNOW', Float(50)),
   Column('SNWD', Float(50)),
   Column('TMAX', Float(50)),
   Column('TMIN', Float(50)),
   Column('ACMH', Float(50)),
   Column('AWND', Float(50)),
   Column('EVAP', Float(50)),
   Column('FRTH', Float(50)),
   Column('PSUN', Float(50)),
   Column('THIC', Float(50)),
   Column('TOBS', Float(50)),
   Column('TSUN', Float(50)),
   Column('WDMV', Float(50)),
   Column('TAVG', Float(50)),
   Column('WT01', Boolean),
   Column('WT02', Boolean),
   Column('WT03', Boolean),
   Column('WT04', Boolean),
   Column('WT05', Boolean),
   Column('WT06', Boolean),
   Column('WT07', Boolean),
   Column('WT08', Boolean),
   Column('WT09', Boolean),
   Column('WT10', Boolean),
   Column('WT11', Boolean),
   Column('WT12', Boolean),
   Column('WT13', Boolean),
   Column('WT14', Boolean),
   Column('WT15', Boolean),
   Column('WT16', Boolean),
   Column('WT17', Boolean),
   Column('WT18', Boolean),
   Column('WT19', Boolean),
   Column('WT21', Boolean),
   Column('WT22', Boolean)
)	
noaa_weather_data_table.create(dbi.engine)
# meta.create_all(dbi.engine)

print('noaa_weather_data table created')

noaa_ghcnd_file_path = './data/noaa_ghcn_aws_data/'
file_name = '2021'

custom_date_parser = lambda x: datetime.strptime(x, "%Y%m%d")

with pd.read_csv(
    f"{noaa_ghcnd_file_path}{file_name}.csv", 
    header=None, 
    usecols=[0,1,2,3],
    names=['station_id', 'date', 'data_type_id', 'data_value'],#, 'measurement_id', 'quality_id', 'source_id', 'observation_time'],
    chunksize=100000, 
    parse_dates=['date'],
    date_parser=custom_date_parser,
    ) as reader:

   chunk_num = 1
   for noaa_ghcn_aws_data_2021 in reader: 

      # Extract country code from station_id
      noaa_ghcn_aws_data_2021['noaa_country_code'] = noaa_ghcn_aws_data_2021.station_id.str[:2]
      # Drop countries that are not in the fao dataset
      noaa_ghcn_aws_data_2021 = noaa_ghcn_aws_data_2021[noaa_ghcn_aws_data_2021.noaa_country_code.isin(noaa_country_codes.noaa_country_code)]
      # Drop unnecessary data types
      noaa_ghcn_aws_data_2021 = noaa_ghcn_aws_data_2021[noaa_ghcn_aws_data_2021.data_type_id.isin(data_ids_to_keep)]
      # Pivot the table.
      noaa_weather_data = noaa_ghcn_aws_data_2021.pivot(index=['noaa_country_code', 'station_id', 'date'], columns='data_type_id', values='data_value')
      # Fill in the weather type columns with zeroes because they are one-hot encoded/binary
      weather_type_cols = noaa_weather_data.columns[noaa_weather_data.columns.str.contains('WT')]
      noaa_weather_data[weather_type_cols] = noaa_weather_data[weather_type_cols].fillna(0)

      noaa_weather_data.reset_index(inplace=True)

      dbi.insert_data(noaa_weather_data, 'noaa_weather_data', if_exists='append')

      print(f'{chunk_num} chunks processed')
      chunk_num += 1

print("Closing connection")
dbi.close_connection()
print("Done")

# 347 chunks = 39min 26sec

noaa_weather_data table created
1 chunks processed
2 chunks processed
3 chunks processed
4 chunks processed
5 chunks processed
6 chunks processed
7 chunks processed
8 chunks processed
9 chunks processed
10 chunks processed
11 chunks processed
12 chunks processed
13 chunks processed
14 chunks processed
15 chunks processed
16 chunks processed
17 chunks processed
18 chunks processed
19 chunks processed
20 chunks processed
21 chunks processed
22 chunks processed
23 chunks processed
24 chunks processed
25 chunks processed
26 chunks processed
27 chunks processed
28 chunks processed
29 chunks processed
30 chunks processed
31 chunks processed
32 chunks processed
33 chunks processed
34 chunks processed
35 chunks processed
36 chunks processed
37 chunks processed
38 chunks processed
39 chunks processed
40 chunks processed
41 chunks processed
42 chunks processed
43 chunks processed
44 chunks processed
45 chunks processed
46 chunks processed
47 chunks processed
48 chunks processed
49 chunks process

### Load noaa countries from db

In [229]:
dbi = DatabaseInterface(db_name='crop_yield_prediction', 
                        user=local_mysql_user, 
                        password=local_mysql_password)

# noaa_countries_df = dbi.db_engine.execute(f"SELECT * FROM noaa_countries")
noaa_countries_df = pd.read_sql_table(
    'noaa_countries',
    con=dbi.engine
)

print("Closing connection")
dbi.close_connection()
print("Done")
noaa_countries_df

Closing connection
Done


,noaa_country_code,noaa_country_name,fao_country_code
0,AC,Antigua and Barbuda,8
1,AE,United Arab Emirates,225
2,AF,Afghanistan,2
3,AG,Algeria,4
4,AJ,Azerbaijan,52
5,AL,Albania,3
6,AM,Armenia,1
7,AO,Angola,7
8,AQ,American Samoa [United States],244
9,AR,Argentina,9


In [104]:
noaa_ghcn_aws_data_2021.head()

,station_id,date,data_type_id,data_value,noaa_country_code
0,AE000041196,2021-01-01,TMAX,278,AE
1,AE000041196,2021-01-01,PRCP,0,AE
2,AE000041196,2021-01-01,TAVG,214,AE
3,AEM00041194,2021-01-01,TMAX,266,AE
4,AEM00041194,2021-01-01,TMIN,178,AE


In [103]:
noaa_ghcn_aws_data_2021.isna().sum()

station_id           0
date                 0
data_type_id         0
data_value           0
noaa_country_code    0
dtype: int64

In [125]:
pd.DataFrame(data_types_to_keep)

,data_type_id,description,units
0,PRCP,Precipitation,tenths of mm
1,SNOW,Snowfall,mm
2,SNWD,Snow depth,mm
3,TMAX,Maximum temperature,tenths of degrees C
4,TMIN,Minimum temperature,tenths of degrees C
5,ACMH,Average cloudiness midnight to midnight from m...,percent
6,AWND,Average daily wind speed,tenths of meters per second
7,EVAP,Evaporation of water from evaporation pan,tenths of mm
8,FRTH,Thickness of frozen ground layer,cm
9,PSUN,Daily percent of possible sunshine,percent


In [133]:
mean_weather_per_country_2021.columns

Index(['noaa_country_code', 'date', 'AWND', 'EVAP', 'PRCP', 'PSUN', 'SNOW',
       'SNWD', 'TAVG', 'THIC', 'TMAX', 'TMIN', 'TOBS', 'TSUN', 'WDMV', 'WT01',
       'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT08', 'WT09', 'WT11'],
      dtype='object', name='data_type_id')

In [ ]:
# Replace NaN values in measurement_id, quality_id and source_id columns to match id descriptions
# noaa_ghcn_aws_data_2021.measurement_id.replace(np.nan, 'None', inplace=True)
# noaa_ghcn_aws_data_2021.quality_id.replace(np.nan, 'None', inplace=True)
# noaa_ghcn_aws_data_2021.source_id.replace(np.nan, 'None', inplace=True)
# noaa_ghcn_aws_data_2021.head()

In [45]:
noaa_ghcn_aws_data_2021.shape

(1000000, 4)

In [46]:
noaa_ghcn_aws_data_2021.element.unique()

array(['TMAX', 'PRCP', 'TAVG', 'TMIN', 'SNWD', 'AWND', 'WDF2', 'WSF2',
       'DATX', 'MDTX', 'DATN', 'MDTN', 'DAPR', 'MDPR', 'SNOW', 'WESF',
       'WESD', 'TOBS', 'WDF5', 'WSF5', 'WT01', 'EVAP', 'PGTM', 'SN32',
       'SX32', 'MNPN', 'MXPN', 'WT11', 'WDMV', 'SN31', 'SX31', 'WT03',
       'SN52', 'SX52', 'SN33', 'SN35', 'SX33', 'SX35', 'WDFG', 'WSFG',
       'WT06', 'WT04', 'THIC', 'SN51', 'SX51', 'SN53', 'SN55', 'SX53',
       'SX55', 'SN36', 'SN56', 'SX36', 'SX56', 'AWDR', 'WSFI', 'WT08',
       'WT02', 'WT09', 'WT05', 'PSUN', 'TSUN', 'DWPR'], dtype=object)

In [ ]:
# Check for multiple measurements at different times
grouped = noaa_ghcn_aws_data_2021.groupby(['station_id', 'date', 'element']).count().sort_values('observation_time')
grouped

In [ ]:
# We can drop the observation_time column because there is only one obervation per day
noaa_ghcn_aws_data_2021.drop('observation_time', axis=1, inplace=True)
noaa_ghcn_aws_data_2021.head()

,station_id,date,element,data_value,noaa_country_code
0,AE000041196,2021-01-01,TMAX,278,AE
1,AE000041196,2021-01-01,PRCP,0,AE
2,AE000041196,2021-01-01,TAVG,214,AE
3,AEM00041194,2021-01-01,TMAX,266,AE
4,AEM00041194,2021-01-01,TMIN,178,AE


## Check for country codes in the NOAA data against the FAO data

In [59]:
noaa_ghcn_aws_data_2021.shape

(1000000, 5)

(996942, 5)

In [72]:
df = noaa_ghcn_aws_data_2021.copy()
df = df[~df.noaa_country_code.isin(new_country_codes['FIPS10-4'])]
df.noaa_country_code.unique()

array(['DA', 'EU', 'JN', 'TE', 'VM', 'WQ'], dtype=object)

In [76]:
new_country_codes[new_country_codes.country_name=='Denmark']

,country_name,FIPS10-4,ISO 3166-1
60,Denmark,DK,DK


In [53]:
# Check for multiple temperature measurements on different soil types and depths
noaa_ghcn_aws_data_2021[noaa_ghcn_aws_data_2021.element.str.contains('SX')]

,station_id,date,element,data_value,noaa_country_code
44545,USC00010655,2021-01-01,SX32,111,US
44615,USC00012813,2021-01-01,SX32,172,US
44640,USC00013251,2021-01-01,SX32,206,US
45556,USC00033428,2021-01-01,SX32,78,US
45590,USC00033821,2021-01-01,SX31,72,US
...,...,...,...,...,...
994576,USC00509793,2021-01-11,SX52,-28,US
994593,USC00509891,2021-01-11,SX52,-17,US
994594,USC00509891,2021-01-11,SX53,-11,US
994595,USC00509891,2021-01-11,SX55,-6,US


In [55]:
noaa_ghcn_aws_data_2021[noaa_ghcn_aws_data_2021.element.str.contains('SX')].groupby(['station_id', 'date']).count().sort_values('element')

element  data_value  noaa_country_code
station_id  date                                              
USC00010655 2021-01-01        1           1                  1
USC00346382 2021-01-04        1           1                  1
            2021-01-03        1           1                  1
            2021-01-02        1           1                  1
            2021-01-01        1           1                  1
...                         ...         ...                ...
USC00218450 2021-01-11        8           8                  8
            2021-01-02        8           8                  8
            2021-01-01        8           8                  8
            2021-01-03        8           8                  8
            2021-01-10        8           8                  8

[1892 rows x 3 columns]

In [57]:
noaa_ghcn_aws_data_2021[noaa_ghcn_aws_data_2021.element.str.contains('SX')].groupby(['station_id', 'date']).mean('data_value').loc['USC00218450']

,data_value
date,
2021-01-01,11.75
2021-01-02,11.75
2021-01-03,11.75
2021-01-04,11.75
2021-01-05,11.75
2021-01-06,11.75
2021-01-07,11.75
2021-01-08,11.75
2021-01-09,10.50


In [ ]:
grouped[grouped.element > 1].sort_values('element') # We need to create all possible soil temp element_ids in case they show up in other years

In [ ]:
noaa_ghcn_aws_data_2021[(noaa_ghcn_aws_data_2021.station_id=='USC00218450')&(noaa_ghcn_aws_data_2021.date=='2021-06-27')][['element', 'data_value']]

In [ ]:
noaa_ghcn_aws_data_2021[noaa_ghcn_aws_data_2021.element=='FMTM'].head(100)

In [ ]:
# Fetch all available data categories
endpoint='datatypes'
querystring = {'limit': 1000, 'datacategoryid': 'TMAX'}
url = f"{base_url}{endpoint}"

datatype = requests.request("GET", url, headers=headers, params=querystring)
pp(datatype.json())

In [ ]:
noaa_ghcn_aws_data_2021[(noaa_ghcn_aws_data_2021.station_id.str.contains('AS')) & (noaa_ghcn_aws_data_2021.date=='2021-01-11')].head(500)

In [ ]:
noaa_ghcn_aws_data_2021[noaa_ghcn_aws_data_2021.element.str.contains('MD')]

In [ ]:
set([item['element_id'] for item in element_ids]) - set(noaa_ghcn_aws_data_2021.element.unique().tolist())

In [ ]:
len(noaa_ghcn_aws_data_2021.element.unique().tolist())

In [ ]:
noaa_ghcn_aws_data_2021[(noaa_ghcn_aws_data_2021.station_id=='USC00218450')&(noaa_ghcn_aws_data_2021.date=='2021-06-27')][['element', 'data_value']]